In [1]:
import sqlalchemy
import uvicorn
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

import re
import time
import datetime

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
def to_lowercase(df):
	df = df.copy()
	df.rename({i:i.lower() for i in df.columns.values}, axis=1, inplace=True)
	return df

def to_snakecase(df):
	snakecase = {i: re.sub(r"[,.;@#?!&$]+\ *", "", i).replace(" ", "_") for i in df}
	df.rename(columns=snakecase, inplace=True)
	return df

# def to_datetime(df, data_format):
	

def get_Users(path="../../../data/users.csv", date_format = "%Y-%m"):
	users = to_snakecase(to_lowercase(pd.read_csv(path)))
	users['birth_year'] = users['birth_year'].astype(str) + '-' + users['birth_month'].astype(str)
	# users['birth_year'] = pd.to_datetime(users['birth_year'].astype(str) + '-' + users['birth_month'].astype(str)).dt.strftime(date_format)
	users = users.drop(columns= ['birth_month'])
	users = users.rename(columns={'birth_year': 'birth'})
	return users


def get_CreditCards(path="../../../data/users.csv", date_format = "%Y-%m"):
	credit_cards = to_snakecase(to_lowercase(pd.read_csv("../../../data/credit_cards.csv")))
	credit_cards['expires'] = pd.to_datetime(credit_cards['expires']).dt.strftime(date_format)
	credit_cards['acct_open_date'] = pd.to_datetime(credit_cards['expires']).dt.strftime(date_format)
	return credit_cards

def get_Transactions(path="../../../data/transactions.csv", date_format = "%Y-%m"):
	transactions = to_snakecase(to_lowercase(pd.read_csv(path)))
	return transactions

users = get_Users()
credit_cards = get_CreditCards()
transactions = get_Transactions()


print(users.columns)
print(credit_cards.columns)
print(transactions.columns)
users

Index(['user', 'person', 'current_age', 'retirement_age', 'birth', 'gender',
       'address', 'apartment', 'city', 'state', 'zipcode', 'latitude',
       'longitude', 'per_capita_income', 'yearly_income', 'total_debt',
       'fico_score', 'num_credit_cards'],
      dtype='object')
Index(['user', 'card_index', 'card_brand', 'card_type', 'card_number',
       'expires', 'cvv', 'has_chip', 'cards_issued', 'credit_limit',
       'acct_open_date', 'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')
Index(['user', 'card', 'year', 'month', 'day', 'time', 'amount', 'use_chip',
       'merchant_name', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'is_fraud'],
      dtype='object')


,user,person,current_age,retirement_age,birth,gender,address,apartment,city,state,zipcode,latitude,longitude,per_capita_income,yearly_income,total_debt,fico_score,num_credit_cards
0,0,Hazel Robinson,53,66,1966-11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,29278,59696,127613,787,5
1,1,Sasha Sadr,53,68,1966-12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,37891,77254,191349,701,5
2,2,Saanvi Lee,81,67,1938-11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,22681,33483,196,698,5
3,3,Everlee Clark,63,63,1957-1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,163145,249925,202328,722,4
4,4,Kyle Peterson,43,70,1976-9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,53797,109687,183855,675,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,Jose Faraday,32,70,1987-7,Male,6577 Lexington Lane,9.0,Freeport,NY,11520,40.65,-73.58,23550,48010,87837,703,3
1996,1996,Ximena Richardson,62,65,1957-11,Female,2 Elm Drive,955.0,Independence,KY,41051,38.95,-84.54,24218,49378,104480,740,4
1997,1997,Annika Russell,47,67,1973-1,Female,276 Fifth Boulevard,NaN,Elizabeth,NJ,7201,40.66,-74.19,15175,30942,71066,779,3
1998,1998,Juelz Roman,66,60,1954-2,Male,259 Valley Boulevard,NaN,Camp Hill,PA,17011,40.24,-76.92,25336,54654,27241,618,1


In [3]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Mapped, mapped_column
from sqlalchemy import create_engine, Column, Integer, String, Double, DateTime, ForeignKey
from sqlalchemy.sql.sqltypes import TIMESTAMP
from sqlalchemy.sql.expression import text

def create_db(user="root", password="msql1234", server="localhost", database="transact"):
    SQLALCHEMY_DATABASE_URL = "mysql+pymysql://{}:{}@{}/{}".format(
        user, password, server, database
    )
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()

    return engine, SessionLocal, Base

engine, SessionLocal, Base = create_db()


db = engine.connect()

db.execute(sqlalchemy.text('''
			SET SESSION SQL_MODE='ALLOW_INVALID_DATES'
		'''))


class Users(Base):
	__tablename__ = 'users'
	user = Column(Integer, primary_key=True, nullable=False)
	# user:Mapped[int] = mapped_column(primary_key=True, nullable=False)
	person = Column(String(32))
	current_age = Column(Integer)
	retirement_age = Column(Integer)
	birth = Column(String(7))
	# birth = Column(TIMESTAMP(timezone=True), server_default=text("now()"))
	gender = Column(String(32))
	address = Column(String(64))
	apartment = Column(Integer)
	city = Column(String(32))
	state = Column(String(32))
	zipcode = Column(String(32))
	latitude = Column(Double)
	longitude = Column(Double)
	per_capita_income = Column(Double)
	yearly_income = Column(Double)
	total_debt = Column(Double)
	fico_score = Column(Double)
	num_credit_cards = Column(Integer)


class Cards(Base):
	__tablename__ = 'credit_cards'
	user = Column(Integer, ForeignKey("users.user", ondelete="CASCADE"), nullable=False)
	# user:Mapped[int] = mapped_column(ForeignKey("users.user", ondelete='CASCADE'))
	card_index = Column(Integer)
	card_brand = Column(String(32))
	card_type = Column(String(32))
	card_number = Column(String(16), primary_key=True, nullable=False)
	# expires  = Column(TIMESTAMP(timezone=True), server_default=text("now()"))
	expires = Column(String(7))
	cvv = Column(String(3))
	has_chip = Column(String(3)) 
	cards_issued = Column(Integer)
	credit_limit = Column(Double)
	# acct_open_date = Column(DateTime)
	acct_open_date = Column(String(7))
	# year_pin_last_changed = Column(DateTime)
	year_pin_last_changed = Column(String(7))
	card_on_dark_web = Column(String(3))

# class Transactions(Base):
# 	pass
	# __tablename__ = "transactions"
	# index = Column(Integer, primary_key=True)
	# user = Column(Integer, ForeignKey(Users.user))
	# card = Column(Integer, ForeignKey(Cards.card_number))
	# # year 
	# # month
	# # day  
	# # time
	# amount = Column(Double)
	# # use_chip
	# # merchant_name
	# # merchant_city
	# # merchant_state
	# # zip
	# # mcc
	# # errors
	# # is_fraud


Base.metadata.create_all(engine)

db.commit()
db.close()

In [7]:
with engine.connect() as db:
	# users.to_sql('users', con=engine, if_exists='append', index=False)
	credit_cards.loc[1:2, :].to_sql('credit_cards', con=engine, if_exists='append', index=False)
	db.commit()
	db.close()

IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`transact`.`credit_cards`, CONSTRAINT `credit_cards_ibfk_1` FOREIGN KEY (`user`) REFERENCES `users` (`user`) ON DELETE CASCADE)')
[SQL: INSERT INTO credit_cards (user, card_index, card_brand, card_type, card_number, expires, cvv, has_chip, cards_issued, credit_limit, acct_open_date, year_pin_last_changed, card_on_dark_web) VALUES (%(user)s, %(card_index)s, %(card_brand)s, %(card_type)s, %(card_number)s, %(expires)s, %(cvv)s, %(has_chip)s, %(cards_issued)s, %(credit_limit)s, %(acct_open_date)s, %(year_pin_last_changed)s, %(card_on_dark_web)s)]
[parameters: [{'user': 0, 'card_index': 1, 'card_brand': 'Visa', 'card_type': 'Debit', 'card_number': 4956965974959986, 'expires': '2020-12', 'cvv': 393, 'has_chip': 'YES', 'cards_issued': 2, 'credit_limit': 21968, 'acct_open_date': '2020-12', 'year_pin_last_changed': 2014, 'card_on_dark_web': 'No'}, {'user': 0, 'card_index': 2, '